In [1]:
#參考資料
#https://deepchem.io/tutorials/

In [9]:
import deepchem as dc
import numpy as np
import tempfile

In [13]:
#使用GraphConv圖卷積網路
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets
test_dataset

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>

In [14]:
test_dataset.to_dataframe()

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.821917,1.0,c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.528916,1.0,C1=Cc2cccc3cccc1c23
...,...,...,...,...
108,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.656304,1.0,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
109,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.743629,1.0,c1ccsc1
110,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.420799,1.0,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
111,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.209570,1.0,Cc1occc1C(=O)Nc2ccccc2


In [20]:
print(type(test_dataset))

<class 'deepchem.data.datasets.DiskDataset'>


In [15]:
#開始訓練模型
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)
model.fit(train_dataset, nb_epoch=100) #執行100次訓練
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

Training set score: {'pearson_r2_score': 0.9196828176474535}
Test set score: {'pearson_r2_score': 0.6188582247319656}


In [18]:
#使用相關係數評估訓練的模型如何，越接近1代表模型學得越好
Training_set_score = model.evaluate(train_dataset, [metric], transformers)
Training_set_score

{'pearson_r2_score': 0.9196828176474535}

In [19]:
Test_set_score = model.evaluate(test_dataset, [metric], transformers)
Test_set_score
#可以看到Training_set_score比Test_set_score高，這是正常的

{'pearson_r2_score': 0.6188582247319656}

In [26]:
#使用訓練的模型從smile ID預測溶解度，預測的是dataset的前10個分子
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility, test_solubility in zip(test_dataset.ids, solubilities, test_dataset.y):
    print(solubility, test_solubility, molecule)

[-1.5866759] [-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[0.5590398] [0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-1.0884441] [-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-1.9287449] [-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-1.4819356] [-0.52891635] C1=Cc2cccc3cccc1c23
[1.9481467] [1.10168349] CC1CO1
[-0.16625102] [-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-1.0831811] [-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-1.3910329] [-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[-0.43014097] [-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 
